# Part 3

#### Set up data from Part 1:

In [121]:
# Set up data from Part 1:
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import requests

# Get data from URL, look at structure with BeautifulSoup.
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
#print(soup.prettify())

# Saw the table in the data above...look at it so we can get column names.
table = soup.find('table',{'class':'wikitable sortable'})
#print(table.prettify())

#Saw the column names, so set up dataframe to hold the data with these column names.
col_names = ['Postal code', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = col_names)

# Read the data from the url into the dataframe
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.head()

# Discard rows with unassigned Boroughs
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)

# Replace unassigned Neighborhoods with name of Borough
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df.Borough

# Replace '/' in aggregated neighborhoods with comma
df.Neighborhood = df.Neighborhood.str.replace(' /',',')

#Reset row index since rows were deleted
df = df.reset_index(drop=True)
#df.head()
df.describe()


,Postal code,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M2P,North York,Downsview
freq,1,24,4


#### Set up data from Part 2
:

In [122]:
# Set up data from Part 2
#Getting lat and long from .csv file...

Geospatial_df=pd.read_csv('http://cocl.us/Geospatial_data')
Geospatial_df.head()
# Need to capitalize Postal Code column in original dataframe (df) to match here...
df.columns = ['Postal Code', 'Borough', 'Neighborhood']
# Set up dataframe to merge the two...
col_names = ['Postal Code', 'Borough', 'Neighborhood']
Geospatial_wLatLon = pd.DataFrame(columns = col_names)

# Merge df and Geospatial_df (validate will ensure Geospatial doesn't contain duplicate Postal Codes)
Geospatial_wLatLon=pd.merge(df, Geospatial_df, on='Postal Code', validate='many_to_one')
Geospatial_wLatLon.shape
Geospatial_wLatLon.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Explore and cluster the neighborhoods in Toronto

In [123]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


In [124]:
# Install folium for maps
!pip install folium # map rendering library
print('folium imported.')

folium imported.


#### Keep Boroughs that have 'Toronto' in their name

In [125]:
TorontoDF = Geospatial_wLatLon[Geospatial_wLatLon.Borough.str.contains('Toronto',case=False)]
TorontoDF = TorontoDF.reset_index(drop=True)
TorontoDF.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### See how many boroughs and neighborhoods we have.

In [126]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorontoDF['Borough'].unique()),
        TorontoDF.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


####  Get lat and log of Toronto

In [127]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of New York with neighborhoods superimposed on top

In [128]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('folium imported')

folium imported


In [131]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoDF['Latitude'], TorontoDF['Longitude'], TorontoDF['Borough'], TorontoDF['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [133]:
# Hiding my credentials before saving and posting to GitHub...

In [ ]:
# Show some neighborhoods.
TorontoDF.head()

In [54]:
# Let's look at a neighborhood in East Toronto... "The Beaches"...row 4
neighborhood_latitude = TorontoDF.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = TorontoDF.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = TorontoDF.loc[4, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


### Now, let's get the top 10 venues that are in The Beaches within a radius of 500 meters.

In [55]:
LIMIT = 10 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=CUIA3OF2C2UBOPRIFH2EK5OLFXJNCJKKHFHRMMW0H3NUS35J&client_secret=GTNAIV5AAJTYUD0GRHDO1MIX0DBNNNHI4PGGSLYAWSUKX2TJ&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=10'

### Send the GET request and examine the results

In [56]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea5f738211536324b1be2c8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

In [57]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Clean the json and structure it into a pandas dataframe.

In [58]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [59]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


## Explore Neighborhoods in Manhattan

### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [89]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [87]:
TorontoDF.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Get venues within 500 meters.

In [90]:
toronto_venues = getNearbyVenues(names=TorontoDF['Neighborhood'],
                                   latitudes=TorontoDF['Latitude'],
                                   longitudes=TorontoDF['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

### Size of the amount...

In [91]:
print(toronto_venues.shape)
toronto_venues.head()

(348, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


### Check how many venues return for each neighborhood

In [92]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,10,10,10,10,10,10
"Brockton, Parkdale Village, Exhibition Place",10,10,10,10,10,10
Business reply mail Processing CentrE,10,10,10,10,10,10
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
Christie,10,10,10,10,10,10
Church and Wellesley,10,10,10,10,10,10
"Commerce Court, Victoria Hotel",10,10,10,10,10,10
Davisville,10,10,10,10,10,10


In [93]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 120 uniques categories.


## Analyzing each neighborhood.

In [94]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [96]:
toronto_onehot.shape

(348, 120)

### Means of the frequency of occurrence of each category, each neighborhood.

In [98]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.1,0.00,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.1,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,Business reply mail Processing CentrE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0.1,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.1,0.0,0.000000,0.0,0.00,0.0,0.0,0.1,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.1,0.1,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.1,0.000000,0.0,0.00,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [99]:
# Size of the above
toronto_grouped.shape

(39, 120)

### Each neighborhood along with the top 5 most common venues

In [100]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0  Farmers Market   0.2
1    Concert Hall   0.1
2        Fountain   0.1
3            Park   0.1
4    Cocktail Bar   0.1


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0  Coffee Shop   0.2
1          Gym   0.1
2          Bar   0.1
3    Pet Store   0.1
4   Restaurant   0.1


----Business reply mail Processing CentrE----
           venue  freq
0     Restaurant   0.1
1  Auto Workshop   0.1
2    Pizza Place   0.1
3  Burrito Place   0.1
4     Skate Park   0.1


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge   0.2
1  Airport Terminal   0.2
2          Boutique   0.1
3           Airport   0.1
4             Plane   0.1


----Central Bay Street----
                        venue  freq
0                 Coffee Shop   0.5
1                         Spa   0.1
2                   Gastropub   0.1
3  Modern European Rest

### Put this into a dataframe for use later...

In [103]:
# Sorting function...
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Farmers Market,Concert Hall,Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Park,Fountain,Museum,Liquor Store,Dim Sum Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Bakery,Pet Store,Café,Restaurant,Bar,Italian Restaurant,Furniture / Home Store,Gym,Food Truck
2,Business reply mail Processing CentrE,Pizza Place,Auto Workshop,Fast Food Restaurant,Farmers Market,Comic Shop,Burrito Place,Restaurant,Brewery,Skate Park,Garden Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport Lounge,Boutique,Rental Car Location,Harbor / Marina,Plane,Airport Food Court,Airport,Cuban Restaurant,Dance Studio
4,Central Bay Street,Coffee Shop,Bubble Tea Shop,Gastropub,Modern European Restaurant,Middle Eastern Restaurant,Spa,Fountain,Food Truck,Garden,Creperie


## Cluster Neighborhoods - k-means, 5 clusters

In [104]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 1, 2, 0, 1, 0, 1, 1], dtype=int32)

In [107]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels1', kmeans.labels_)

toronto_merged = TorontoDF

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,2,Breakfast Spot,Coffee Shop,Gym / Fitness Center,Restaurant,Distribution Center,Park,Historic Site,Bakery,Spa,Food
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,2,Coffee Shop,Sushi Restaurant,Creperie,Mexican Restaurant,Distribution Center,Italian Restaurant,Beer Bar,Park,Food,Cosmetics Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,0,Café,Clothing Store,Tea Room,Coffee Shop,Music Venue,Plaza,Burrito Place,Comic Shop,Theater,Food Truck
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,2,Coffee Shop,Gastropub,Gym,Creperie,Restaurant,Middle Eastern Restaurant,Food Truck,Japanese Restaurant,Cosmetics Shop,Food & Drink Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,2,Trail,Pub,Health Food Store,Wine Bar,Eastern European Restaurant,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store


In [ ]:
Visualize the clusters...

In [134]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters...

### Cluster 1

In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,0,Café,Clothing Store,Tea Room,Coffee Shop,Music Venue,Plaza,Burrito Place,Comic Shop,Theater,Food Truck
7,Downtown Toronto,0,0,Café,Grocery Store,Candy Store,Italian Restaurant,Diner,Restaurant,Coffee Shop,Department Store,Eastern European Restaurant,Dessert Shop
8,Downtown Toronto,0,0,Concert Hall,Steakhouse,Vegetarian / Vegan Restaurant,Hotel,Speakeasy,Café,Restaurant,Plaza,Gym / Fitness Center,Diner
9,West Toronto,0,0,Bakery,Middle Eastern Restaurant,Bank,Music Venue,Grocery Store,Gym / Fitness Center,Café,Brewery,Bar,Donut Shop
13,Downtown Toronto,0,0,Café,Gym / Fitness Center,Bakery,Coffee Shop,Restaurant,Beer Bar,Tea Room,Gym,Pub,Distribution Center
16,Downtown Toronto,0,0,Café,Gym / Fitness Center,Coffee Shop,Museum,Restaurant,Tea Room,Gym,Pub,Bakery,Department Store
22,West Toronto,0,0,Gastropub,Arts & Crafts Store,Italian Restaurant,Park,Speakeasy,Bar,Flea Market,Café,Music Venue,Furniture / Home Store
24,Central Toronto,0,0,Coffee Shop,BBQ Joint,Middle Eastern Restaurant,Café,Burger Joint,Donut Shop,Park,Indian Restaurant,American Restaurant,Vegetarian / Vegan Restaurant
28,West Toronto,0,0,Sushi Restaurant,Fish & Chips Shop,Coffee Shop,Bookstore,Italian Restaurant,Tea Room,Café,Food,Burrito Place,Pub
30,Downtown Toronto,0,0,Café,Vietnamese Restaurant,Dessert Shop,Organic Grocery,Mexican Restaurant,Arts & Crafts Store,Bakery,Dog Run,Distribution Center,Diner


### Cluster 2

In [111]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,1,1,Farmers Market,Concert Hall,Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Park,Fountain,Museum,Liquor Store,Dim Sum Restaurant
10,Downtown Toronto,1,1,Performing Arts Venue,Supermarket,Park,Plaza,Lake,Salad Place,Hotel,Sporting Goods Shop,Skating Rink,Fish & Chips Shop
11,West Toronto,1,1,Wine Bar,Beer Store,Ice Cream Shop,Korean Restaurant,Cuban Restaurant,New American Restaurant,Cocktail Bar,Brewery,Pizza Place,Asian Restaurant
12,East Toronto,1,1,Greek Restaurant,Ice Cream Shop,Yoga Studio,Brewery,Italian Restaurant,Cosmetics Shop,Fruit & Vegetable Store,French Restaurant,Garden,Creperie
15,East Toronto,1,1,Pet Store,Liquor Store,Italian Restaurant,Park,Ice Cream Shop,Fast Food Restaurant,Fish & Chips Shop,Gym,Brewery,Sushi Restaurant
17,East Toronto,1,1,Pet Store,Bakery,Coffee Shop,Sandwich Place,Italian Restaurant,Ice Cream Shop,Fish Market,Bookstore,Gay Bar,Food & Drink Shop
18,Central Toronto,1,1,Park,Bus Line,Swim School,Dim Sum Restaurant,Wine Bar,Eastern European Restaurant,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
20,Central Toronto,1,1,Park,Gym,Sandwich Place,Hotel,Department Store,Food & Drink Shop,Breakfast Spot,Dance Studio,Cuban Restaurant,Donut Shop
21,Central Toronto,1,1,Trail,Sushi Restaurant,Mexican Restaurant,Jewelry Store,Wine Bar,Donut Shop,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
26,Central Toronto,1,1,Italian Restaurant,Dessert Shop,Sushi Restaurant,Pizza Place,Café,Seafood Restaurant,Indian Restaurant,Coffee Shop,Food,Distribution Center


### Cluster 3

In [112]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,2,Breakfast Spot,Coffee Shop,Gym / Fitness Center,Restaurant,Distribution Center,Park,Historic Site,Bakery,Spa,Food
1,Downtown Toronto,2,2,Coffee Shop,Sushi Restaurant,Creperie,Mexican Restaurant,Distribution Center,Italian Restaurant,Beer Bar,Park,Food,Cosmetics Shop
3,Downtown Toronto,2,2,Coffee Shop,Gastropub,Gym,Creperie,Restaurant,Middle Eastern Restaurant,Food Truck,Japanese Restaurant,Cosmetics Shop,Food & Drink Shop
4,East Toronto,2,2,Trail,Pub,Health Food Store,Wine Bar,Eastern European Restaurant,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store
6,Downtown Toronto,2,2,Coffee Shop,Bubble Tea Shop,Gastropub,Modern European Restaurant,Middle Eastern Restaurant,Spa,Fountain,Food Truck,Garden,Creperie
14,West Toronto,2,2,Coffee Shop,Bakery,Pet Store,Café,Restaurant,Bar,Italian Restaurant,Furniture / Home Store,Gym,Food Truck
23,Central Toronto,2,2,Yoga Studio,Spa,Diner,Mexican Restaurant,Dessert Shop,Coffee Shop,Chinese Restaurant,Restaurant,Salon / Barbershop,Clothing Store
25,West Toronto,2,2,Gift Shop,Coffee Shop,Restaurant,Dog Run,Italian Restaurant,Dessert Shop,Movie Theater,Cuban Restaurant,Eastern European Restaurant,Department Store
31,Central Toronto,2,2,Coffee Shop,Liquor Store,Restaurant,Bank,Sports Bar,Pub,Supermarket,Sushi Restaurant,American Restaurant,Department Store


### Cluster 4

In [113]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,3,3,Park,Trail,Playground,Eastern European Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store
33,Downtown Toronto,3,3,Park,Trail,Playground,Eastern European Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store


### Cluster 5

In [114]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,4,Garden,Music Venue,Wine Bar,Eastern European Restaurant,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store,Dessert Shop
